# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 13 2022 11:35AM,246277,18,6961284,"Blinc International, Inc.-Blinc-ecom",Released
1,Sep 13 2022 11:35AM,246276,10,CLI00381.1,"CLINUVEL, Inc.",Released
2,Sep 13 2022 11:35AM,246276,10,CLI00380.1,"CLINUVEL, Inc.",Released
3,Sep 13 2022 11:09AM,246271,10,EYE42987,Eye Pharma Inc,Released
4,Sep 13 2022 10:58AM,246269,19,MSP213917,"Methapharm, Inc.",Released
5,Sep 13 2022 10:58AM,246269,19,MSP213918,"Methapharm, Inc.",Released
6,Sep 13 2022 10:58AM,246269,19,MSP213921,"Methapharm, Inc.",Released
7,Sep 13 2022 10:57AM,246267,20,8516583,"ACI Healthcare USA, Inc.",Released
8,Sep 13 2022 10:57AM,246267,20,8516584,"ACI Healthcare USA, Inc.",Released
9,Sep 13 2022 10:57AM,246267,20,8516696,"ACI Healthcare USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
19,246268,Released,2
20,246269,Released,3
21,246271,Released,1
22,246276,Released,2
23,246277,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
246268,NaN,NaN,2.0
246269,NaN,NaN,3.0
246271,NaN,NaN,1.0
246276,NaN,NaN,2.0
246277,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246064,0.0,0.0,1.0
246156,0.0,2.0,3.0
246190,1.0,0.0,1.0
246194,0.0,0.0,1.0
246214,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246064,0,0,1
246156,0,2,3
246190,1,0,1
246194,0,0,1
246214,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246064,0,0,1
1,246156,0,2,3
2,246190,1,0,1
3,246194,0,0,1
4,246214,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246064,,,1
1,246156,,2,3
2,246190,1,,1
3,246194,,,1
4,246214,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 13 2022 11:35AM,246277,18,"Blinc International, Inc.-Blinc-ecom"
1,Sep 13 2022 11:35AM,246276,10,"CLINUVEL, Inc."
3,Sep 13 2022 11:09AM,246271,10,Eye Pharma Inc
4,Sep 13 2022 10:58AM,246269,19,"Methapharm, Inc."
7,Sep 13 2022 10:57AM,246267,20,"ACI Healthcare USA, Inc."
37,Sep 13 2022 10:51AM,246226,21,"NBTY Global, Inc."
38,Sep 13 2022 10:51AM,246268,19,"GUSA Granules USA, Inc."
40,Sep 13 2022 10:44AM,246265,10,"Methapharm, Inc."
52,Sep 13 2022 10:22AM,246262,10,"Nextsource Biotechnology, LLC"
53,Sep 13 2022 10:10AM,246260,10,Bio-PRF


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 13 2022 11:35AM,246277,18,"Blinc International, Inc.-Blinc-ecom",,,1
1,Sep 13 2022 11:35AM,246276,10,"CLINUVEL, Inc.",,,2
2,Sep 13 2022 11:09AM,246271,10,Eye Pharma Inc,,,1
3,Sep 13 2022 10:58AM,246269,19,"Methapharm, Inc.",,,3
4,Sep 13 2022 10:57AM,246267,20,"ACI Healthcare USA, Inc.",,4,26
5,Sep 13 2022 10:51AM,246226,21,"NBTY Global, Inc.",,1,
6,Sep 13 2022 10:51AM,246268,19,"GUSA Granules USA, Inc.",,,2
7,Sep 13 2022 10:44AM,246265,10,"Methapharm, Inc.",,,12
8,Sep 13 2022 10:22AM,246262,10,"Nextsource Biotechnology, LLC",,,1
9,Sep 13 2022 10:10AM,246260,10,Bio-PRF,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 13 2022 11:35AM,246277,18,"Blinc International, Inc.-Blinc-ecom",1,,
1,Sep 13 2022 11:35AM,246276,10,"CLINUVEL, Inc.",2,,
2,Sep 13 2022 11:09AM,246271,10,Eye Pharma Inc,1,,
3,Sep 13 2022 10:58AM,246269,19,"Methapharm, Inc.",3,,
4,Sep 13 2022 10:57AM,246267,20,"ACI Healthcare USA, Inc.",26,4,
5,Sep 13 2022 10:51AM,246226,21,"NBTY Global, Inc.",,1,
6,Sep 13 2022 10:51AM,246268,19,"GUSA Granules USA, Inc.",2,,
7,Sep 13 2022 10:44AM,246265,10,"Methapharm, Inc.",12,,
8,Sep 13 2022 10:22AM,246262,10,"Nextsource Biotechnology, LLC",1,,
9,Sep 13 2022 10:10AM,246260,10,Bio-PRF,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 13 2022 11:35AM,246277,18,"Blinc International, Inc.-Blinc-ecom",1,,
1,Sep 13 2022 11:35AM,246276,10,"CLINUVEL, Inc.",2,,
2,Sep 13 2022 11:09AM,246271,10,Eye Pharma Inc,1,,
3,Sep 13 2022 10:58AM,246269,19,"Methapharm, Inc.",3,,
4,Sep 13 2022 10:57AM,246267,20,"ACI Healthcare USA, Inc.",26,4,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 13 2022 11:35AM,246277,18,"Blinc International, Inc.-Blinc-ecom",1.0,NaN,NaN
1,Sep 13 2022 11:35AM,246276,10,"CLINUVEL, Inc.",2.0,NaN,NaN
2,Sep 13 2022 11:09AM,246271,10,Eye Pharma Inc,1.0,NaN,NaN
3,Sep 13 2022 10:58AM,246269,19,"Methapharm, Inc.",3.0,NaN,NaN
4,Sep 13 2022 10:57AM,246267,20,"ACI Healthcare USA, Inc.",26.0,4.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 13 2022 11:35AM,246277,18,"Blinc International, Inc.-Blinc-ecom",1.0,0.0,0.0
1,Sep 13 2022 11:35AM,246276,10,"CLINUVEL, Inc.",2.0,0.0,0.0
2,Sep 13 2022 11:09AM,246271,10,Eye Pharma Inc,1.0,0.0,0.0
3,Sep 13 2022 10:58AM,246269,19,"Methapharm, Inc.",3.0,0.0,0.0
4,Sep 13 2022 10:57AM,246267,20,"ACI Healthcare USA, Inc.",26.0,4.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1723817,33.0,0.0,0.0
102,246194,1.0,0.0,0.0
15,246227,74.0,45.0,0.0
16,492370,3.0,3.0,0.0
18,246277,1.0,0.0,0.0
19,1231230,8.0,0.0,1.0
20,246267,26.0,4.0,0.0
21,492290,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1723817,33.0,0.0,0.0
1,102,246194,1.0,0.0,0.0
2,15,246227,74.0,45.0,0.0
3,16,492370,3.0,3.0,0.0
4,18,246277,1.0,0.0,0.0
5,19,1231230,8.0,0.0,1.0
6,20,246267,26.0,4.0,0.0
7,21,492290,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,33.0,0.0,0.0
1,102,1.0,0.0,0.0
2,15,74.0,45.0,0.0
3,16,3.0,3.0,0.0
4,18,1.0,0.0,0.0
5,19,8.0,0.0,1.0
6,20,26.0,4.0,0.0
7,21,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,33.0
1,102,Released,1.0
2,15,Released,74.0
3,16,Released,3.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,15,16,18,19,20,21
Status,,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
Executing,0.0,0.0,45.0,3.0,0.0,0.0,4.0,1.0
Released,33.0,1.0,74.0,3.0,1.0,8.0,26.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,15,16,18,19,20,21
0,Completed,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,Executing,0.0,0.0,45.0,3.0,0.0,0.0,4.0,1.0
2,Released,33.0,1.0,74.0,3.0,1.0,8.0,26.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,15,16,18,19,20,21
0,Completed,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,Executing,0.0,0.0,45.0,3.0,0.0,0.0,4.0,1.0
2,Released,33.0,1.0,74.0,3.0,1.0,8.0,26.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()